## Extracting the Coordinates

In [1]:
import requests
import json

In [2]:
# Overpass API endpoint
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

In [3]:
# Example query: All cycling routes in WEST COAST USA (bicycle routes)
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#
query = """
[out:json][timeout:900];
(
  relation["route"="bicycle"](32.12,-125.15,42.04,-114.10);
);
out geom;
"""
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#

In [4]:
# Make the request
response = requests.get(url, params={'data': query})

In [5]:
# Check success
if response.status_code == 200:
    data = response.json()
    print("✅ Success:", len(data['elements']), "routes found")
else:
    print("❌ Error:", response.status_code)

✅ Success: 926 routes found


In [6]:
# Save response to file
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("West_Coast_cycle_routes.json", "w") as f:
    json.dump(data, f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [7]:
import pandas as pd

# Create empty list to store results
routes_summary = []

# Loop through each route in the dataset
for route in data['elements']:
    if route['type'] != 'relation':
        continue  # skip non-route elements

    coords = []
    for member in route.get('members', []):
        if 'geometry' in member:
            segment_coords = [(pt['lon'], pt['lat']) for pt in member['geometry']]
            coords.extend(segment_coords)

    if coords:
        start_coord = coords[0]
        end_coord = coords[-1]
        routes_summary.append({
            'id': route.get('id'),
            'name': route.get('tags', {}).get('name', 'Unnamed route'),
            'start_lon': start_coord[0],
            'start_lat': start_coord[1],
            'end_lon': end_coord[0],
            'end_lat': end_coord[1],
            'num_points': len(coords)
        })

In [8]:
# Convert results to DataFrame
routes_df = pd.DataFrame(routes_summary)

In [9]:
routes_df = routes_df.sort_values(by='num_points', ascending=False)

# Extracting the features UK First 2,000

In [10]:
import os
AK = os.getenv("AK")

In [11]:
import openrouteservice
import pandas as pd
import json
import time
import os

In [12]:
# Initialize client
client = openrouteservice.Client(key=AK)  # Replace AK with your ORS API key

In [13]:
results = []
save_interval = 10

In [14]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
save_path = "West_Coast_cycle_routes.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [15]:
test_df = routes_df.iloc[0000:1000]

In [16]:
test_df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
82,325779,San Francisco Bay Trail,-122.305809,37.574967,-122.286698,38.211741,35941
877,17368186,Unnamed route,-117.084037,34.857574,-118.024072,36.383192,25913
736,12891023,Unnamed route,-122.485954,37.661864,-122.506829,37.735652,22114
737,12891024,Unnamed route,-124.208266,41.998407,-122.475161,37.807426,22050
903,19322682,Oregon Timber Trail,-121.141359,43.000659,-121.568447,43.988783,21950
...,...,...,...,...,...,...,...
357,2301191,McCoy Creek Bicycle and Pedestrian Path,-122.010747,38.243606,-122.006640,38.248573,3
827,15951002,Unnamed route,-122.235281,37.801714,-122.233882,37.802838,3
731,12737850,Unnamed route,-117.694289,33.607331,-117.694399,33.607320,3
899,18864739,Unnamed route,-122.121753,36.970178,-122.121769,36.970398,2


In [18]:
for i, row in test_df.iterrows():
    start = (row['start_lon'], row['start_lat'])
    end = (row['end_lon'], row['end_lat'])

    try:
        route = client.directions(
            coordinates=[start, end],
            profile='cycling-regular',
            format='geojson',
            elevation=True,
            instructions=True,
            extra_info=['surface', 'waytype', 'waycategory', 'steepness'],
        )



        # --- SAVE RAW ORS RESPONSE FOR DEBUGGING ---
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        raw_dir = "raw_ors_responses_West_Coast"
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        os.makedirs(raw_dir, exist_ok=True)

        raw_filename = f"{raw_dir}/route_{row['id']}.json"

        with open(raw_filename, "w") as raw_file:
            json.dump(route, raw_file, indent=2)
        # -------------------------------------------


        props = route['features'][0]['properties']
        summary = props['summary']
        extras = props.get('extras', {})
        segments = props['segments']
        steps = segments[0]['steps']
        turn_steps = [s for s in steps if s['type'] in {0,1,2,3,4,5,6,7}]
        turns = len(turn_steps)
        steps = len(steps)

        results.append({
            'id': row['id'],
            'name': row['name'],
            'distance_m': summary.get('distance'),
            'duration_s': summary.get('duration'),
            'ascent_m': [props['ascent']],
            'descent_m': [props['descent']],
            'steps': steps,
            'turns': turns,
            'surface': extras.get('surface', {}).get('values', []),
            'waytype': extras.get('waytype', {}).get('values', []),
            'waycategory': extras.get('waycategory', {}).get('values', []),
            'steepness': extras.get('steepness', {}).get('values', []),
        })

        print(f"✅ Route {i+1} processed successfully")
        time.sleep(1.5)  # rate-limit protection

        # Save partial file
        if (i + 1) % save_interval == 0:
            with open(save_path, "w") as f:
                json.dump(results, f, indent=2)
            print(f"💾 Saved partial results after {i+1} routes")

    except Exception as e:
        print(f"❌ Error on route {row['id']}: {e}")
        continue

✅ Route 83 processed successfully
✅ Route 878 processed successfully
✅ Route 737 processed successfully
✅ Route 738 processed successfully
✅ Route 904 processed successfully
✅ Route 471 processed successfully
✅ Route 888 processed successfully
✅ Route 881 processed successfully
✅ Route 664 processed successfully
✅ Route 110 processed successfully
💾 Saved partial results after 110 routes
✅ Route 507 processed successfully
✅ Route 736 processed successfully
✅ Route 739 processed successfully
✅ Route 27 processed successfully
✅ Route 487 processed successfully
✅ Route 520 processed successfully
💾 Saved partial results after 520 routes
✅ Route 782 processed successfully
✅ Route 395 processed successfully
✅ Route 77 processed successfully
✅ Route 883 processed successfully
✅ Route 884 processed successfully
✅ Route 604 processed successfully
✅ Route 756 processed successfully
✅ Route 60 processed successfully
💾 Saved partial results after 60 routes
✅ Route 381 processed successfully
✅ Route

In [19]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
final_path = "ors_route_results_West_Coast.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [20]:
with open(final_path, "w") as f:
    json.dump(results, f, indent=2)

In [23]:
print(f"\n🎉 Finished run — processed {len(results)} routes.")
print(f"📁 Full output saved to {final_path}")


🎉 Finished run — processed 932 routes.
📁 Full output saved to ors_route_results_West_Coast.json


In [24]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("ors_route_results_West_Coast.json", "r") as f:
    results = json.load(f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [25]:
# Convert to a DataFrame
df = pd.DataFrame(results)

In [26]:
# Display the first few rows
display(df.head(1))

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,325779,San Francisco Bay Trail,105654.1,22928.2,[1208.8],[1209.8],119,109,"[[0, 2, 3], [2, 78, 0], [78, 96, 3], [96, 103,...","[[0, 19, 6], [19, 25, 2], [25, 26, 1], [26, 27...","[[0, 920, 0], [920, 945, 8], [945, 2390, 0]]","[[0, 598, 0], [598, 599, 4], [599, 608, -2], [..."


In [27]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
df.to_csv("West_Coast_USA.csv", index=False)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [28]:
df

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,325779,San Francisco Bay Trail,105654.1,22928.2,[1208.8],[1209.8],119,109,"[[0, 2, 3], [2, 78, 0], [78, 96, 3], [96, 103,...","[[0, 19, 6], [19, 25, 2], [25, 26, 1], [26, 27...","[[0, 920, 0], [920, 945, 8], [945, 2390, 0]]","[[0, 598, 0], [598, 599, 4], [599, 608, -2], [..."
1,17368186,Unnamed route,222070.0,44514.4,[1551.6],[1159.7],15,12,"[[0, 17, 3], [17, 21, 0], [21, 67, 3], [67, 78...","[[0, 17, 3], [17, 80, 2], [80, 568, 1], [568, ...","[[0, 80, 0], [80, 158, 1], [158, 190, 0], [190...","[[0, 28, -1], [28, 137, 0], [137, 138, 1], [13..."
2,12891023,Unnamed route,9533.2,1906.6,[173.3],[280.3],9,4,"[[0, 9, 3], [9, 47, 0], [47, 57, 3], [57, 61, ...","[[0, 20, 2], [20, 48, 3], [48, 49, 2], [49, 13...","[[0, 162, 0]]","[[0, 7, 2], [7, 13, 4], [13, 20, 2], [20, 28, ..."
3,12891024,Unnamed route,677243.8,135834.4,[14522.4],[14491.4],133,110,"[[0, 85, 3], [85, 87, 0], [87, 174, 3], [174, ...","[[0, 14, 1], [14, 250, 2], [250, 256, 1], [256...","[[0, 2043, 0], [2043, 2059, 1], [2059, 2368, 0...","[[0, 140, 0], [140, 184, -1], [184, 575, 0], [..."
4,19322682,Oregon Timber Trail,166230.8,37790.3,[1544.8],[1260.8],78,69,"[[0, 121, 1], [121, 125, 3], [125, 136, 2], [1...","[[0, 121, 2], [121, 125, 1], [125, 146, 3], [1...","[[0, 2061, 0]]","[[0, 10, -1], [10, 56, 1], [56, 75, -1], [75, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
927,2301191,McCoy Creek Bicycle and Pedestrian Path,659.4,131.9,[4.0],[0.0],2,0,"[[0, 2, 4]]","[[0, 2, 6]]","[[0, 2, 0]]","[[0, 2, 0]]"
928,15951002,Unnamed route,175.3,35.1,[0.0],[12.0],2,0,"[[0, 2, 3]]","[[0, 2, 3]]","[[0, 2, 0]]","[[0, 2, -2]]"
929,12737850,Unnamed route,10.3,2.1,[0.0],[0.0],2,0,"[[0, 1, 0]]","[[0, 1, 6]]","[[0, 1, 0]]",[]
930,18864739,Unnamed route,24.5,14.7,[0.0],[0.0],2,0,"[[0, 1, 0]]","[[0, 1, 3]]","[[0, 1, 0]]",[]
